In [48]:
import hashlib
from elasticsearch import Elasticsearch
#https://www.elastic.co/blog/how-to-find-and-remove-duplicate-documents-in-elasticsearch
es = Elasticsearch(["localhost:9200"])
dict_of_duplicate_docs = {}
# The following line defines the fields that will be
# used to determine if a document is a duplicate
keys_to_include_in_hash = ["url", "title"]
# Process documents returned by the current search/scroll
def populate_dict_of_duplicate_docs(hits):
    for item in hits:
        combined_key = ""
        for mykey in keys_to_include_in_hash:
            combined_key += str(item['_source'][mykey])
        _id = item["_id"]
        hashval = hashlib.md5(combined_key.encode('utf-8')).digest()
        # If the hashval is new, then we will create a new key
        # in the dict_of_duplicate_docs, which will be
        # assigned a value of an empty array.
        # We then immediately push the _id onto the array.
        # If hashval already exists, then
        # we will just push the new _id onto the existing array
        dict_of_duplicate_docs.setdefault(hashval, []).append(_id)
# Loop over all documents in the index, and populate the
# dict_of_duplicate_docs data structure.
async def scroll_over_all_docs(index):
    data = es.search(index=index, scroll='1m',  body={"query": {"match_all": {}}})
    # Get the scroll ID
    sid = data['_scroll_id']
    scroll_size = len(data['hits']['hits'])
    # Before scroll, process current batch of hits
    populate_dict_of_duplicate_docs(data['hits']['hits'])
    while scroll_size > 0:
        data = es.scroll(scroll_id=sid, scroll='2m')
        # Process current batch of hits
        populate_dict_of_duplicate_docs(data['hits']['hits'])
        # Update the scroll ID
        sid = data['_scroll_id']
        # Get the number of results that returned in the last scroll
        scroll_size = len(data['hits']['hits'])
    
    return dict_of_duplicate_docs
def loop_over_hashes_and_remove_duplicates(index, dict):
    # Search through the hash of doc values to see if any
    # duplicate hashes have been found
    for hashval, array_of_ids in dict.items():
      if len(array_of_ids) > 1:
        print("********** Duplicate docs hash=%s **********" % hashval)
        # Get the documents that have mapped to the current hashval
        matching_docs = es.mget(index=index, body={"ids": array_of_ids})
        #for doc in matching_docs['docs']:
        for num, doc in enumerate(matching_docs['docs'], start=2):
            # In this example, we just print the duplicate docs.
            # This code could be easily modified to delete duplicates
            # here instead of printing them
            #print("doc=%s\n" % doc)
            #DELETE /<index>/_
            # doc/<_id>
            print(doc)
                # res = es.delete(index=index, id=doc["_id"], doc_type='doc')
            res = es.delete(index, doc["_id"])
            print(res)
            # if num > 2:
            #     print(doc)
            #     # res = es.delete(index=index, id=doc["_id"], doc_type='doc')
            #     res = es.delete(index, doc["_id"])
            #     print(res)

In [49]:
async def deleteIndex(index):
    try:
        print(index)
        dict = await scroll_over_all_docs(index)
        if(dict != None):
            res = await loop_over_hashes_and_remove_duplicates(index, dict)
            print(res)
            dict_of_duplicate_docs = {}
    except Exception as e:
        print('error')
        print(e)

In [50]:
import re

for index in es.indices.get('*'):
    if(re.match(r'^[e|i]+[n|s]', index)):
        await deleteIndex(index)
        #tasks = scroll_over_all_docs(index)
        #await asyncio.wait(tasks)
        #loop_over_hashes_and_remove_duplicates(index)
        

enfj
error
object NoneType can't be used in 'await' expression
enfp
error
object NoneType can't be used in 'await' expression
entj
error
object NoneType can't be used in 'await' expression
entp
error
object dict can't be used in 'await' expression
esfj
error
object dict can't be used in 'await' expression
esfp
error
object dict can't be used in 'await' expression
estj
error
object dict can't be used in 'await' expression
estp
error
object dict can't be used in 'await' expression
infj
error
object dict can't be used in 'await' expression
infp
error
object dict can't be used in 'await' expression
intj
error
object dict can't be used in 'await' expression
intp
error
object dict can't be used in 'await' expression
isfj
error
object dict can't be used in 'await' expression
isfp
error
object dict can't be used in 'await' expression
istj
error
object dict can't be used in 'await' expression
istp
error
object dict can't be used in 'await' expression
